In [1]:
# run forward simulation for all the selected events
# set global parameters
import numpy as np
from glob import glob
from os.path import join
import sys
import sh
import os

curdir = '/work/05880/tg851792/stampede2/WUS/initial_forward/'    # absolute path
os.chdir(curdir)
model_dir = "/work/05880/tg851792/stampede2/WUS/initial_model/"   # meshed model storage
print(curdir)
cmt_dir = "/work/05880/tg851792/stampede2/WUS/initial_model/DATA/CMTs"

def init_structure(base, cmtfiles, ref):
    """
    base: base directory
    ref: where the specfem3d code locates
    cmtfiles: the directory to store cmtfiles
    """
    # base dir
    sh.mkdir("-p", base)
    # work dir
    sh.mkdir("-p", join(base, "work"))

    # get work directories' list
    dirs_raw = glob(join(cmtfiles, "*"))
    evtnm = [i.split("/")[-1].split("_")[-1] for i in dirs_raw]
    
    dirs = evtnm
    
    # mkdir of the working directories
    for dir in dirs:
        sh.mkdir("-p", join(base, "work", dir))

    # ln the bin and DATABASE to work directories
    for dir in dirs:
        sh.ln("-s", join(ref, "bin"), join(base, "work", dir, "bin"))
        sh.ln("-s", join(ref, "DATABASES_MPI"), join(base, "work", dir, "DATABASES_MPI"))

    # mkdir DATA and OUTPUT_FILES in work directory
    for dir in dirs:
        sh.mkdir(join(base, "work", dir, "DATA"))
        sh.mkdir(join(base, "work", dir, "OUTPUT_FILES"))

    # cp and ln files in DATA
    toln = ["cemRequest", "crust1.0", "crust2.0",
            "crustmap", "epcrust", "eucrust-07", "GLL", "heterogen", 
            "Lebedev_sea99", "Montagner_model", "old", "PPM", "QRFSI12", 
            "s20rts", "s362ani", "s40rts", "Simons_model", "topo_bathy", "Zhao_JP_model"]
    for dir in dirs:
        sh.cp(join(cmtfiles, "CMTSOLUTION_"+dir), join(
            base, "work", dir, "DATA", "CMTSOLUTION"))
        sh.cp(join(ref, "DATA", "Par_file"),
              join(base, "work", dir, "DATA", "Par_file"))
        sh.cp(join(ref, "DATA", "STAs", "STATIONS_"+dir+"_in"),
              join(base, "work", dir, "DATA", "STATIONS"))
        for lnfile in toln:
            sh.ln("-s", join(ref, "DATA", lnfile),
                  join(base, "work", dir, "DATA", lnfile))

        # ln in workfiles
        toln_work = ["utils"]
        for lnfile in toln_work:
            sh.ln("-s", join(ref, lnfile),
                join(base, "work", dir, lnfile))

    # cp values from mesher and addressing in OUTPUT_FILES to WORK dirs
    for dir in dirs:
        sh.cp(join(ref, "OUTPUT_FILES", "addressing.txt"),
              join(base, "work", dir, "OUTPUT_FILES"))
        sh.cp(join(ref, "OUTPUT_FILES", "values_from_mesher.h"),
              join(base, "work", dir, "OUTPUT_FILES"))
    
    print("structure set up")
    return 0
        
        
# set up calculation structure
# progdir should use the absolute path, already built and meshed, 
# with the DATABASES_MPI linked to $SCRATCH

basedir = "/work/05880/tg851792/stampede2/WUS/initial_forward" # the working dir of the depth perturbation
progdir = "/work/05880/tg851792/stampede2/WUS/initial_model"   

#init_structure(basedir,cmt_dir,progdir)

/work/05880/tg851792/stampede2/WUS/initial_forward/


In [2]:
# functions to submit the job
import math
from slurmpy import Slurm
def get_scripts(thedirs,N_each,N_iter,N_total,nproc):
    """
    input: 
    thedirs: directories for run, each event
    N_each: number of simulations at each iteration ###N_each*nproc <= 2760###
    N_iter: iterations
    N_total: total number of simulations
    nproc: cores per simulation
    """
    result = ""
    curdir = os.path.abspath(".")
    # for xspecfem3D
    result += f"echo 'start xspecfem3D'; "
    for iiter in range(N_iter):
        result += f"echo 'start iteration {iiter}'; "
        # to see if it is the last iter
        if(iiter == N_iter-1):
            n_remain = N_total - N_each * iiter
            for ieach in range(n_remain):
                # ievent
                ievent = iiter * N_each + ieach
                ievent_dir = thedirs[ievent]
                result += f"cd {ievent_dir}; "
                # if n_remain-1
                if(ieach == n_remain-1):
                    inc = ieach*nproc
                    result += f"ibrun -n {nproc} -o {inc} ./bin/xspecfem3D; "
                else:
                    inc = ieach*nproc
                    result += f"ibrun -n {nproc} -o {inc} ./bin/xspecfem3D & "
                # cd
                result += f"cd {curdir}; "
        else:  # normal iters
            for ieach in range(N_each):
                # ievent
                ievent = iiter*N_each+ieach
                ievent_dir = thedirs[ievent]
                # cd
                result += f"cd {ievent_dir}; "
                # if N_each-1
                if(ieach == N_each-1):
                    inc = ieach*nproc
                    result += f"ibrun -n {nproc} -o {inc} ./bin/xspecfem3D; "
                else:
                    inc = ieach*nproc
                    result += f"ibrun -n {nproc} -o {inc} ./bin/xspecfem3D & "
                # cd
                result += f"cd {curdir}; "
        result += f"wait; "
        result += f"echo 'end iteration {iiter}'; "

    return result

def submit_job(thecommand,N_each,nproc,esti_time):
    ## please change the time and the account accordingly
    ## safe time is 2*(simulation time for one event)*(N_iter)
    s = Slurm("sync", {"nodes": math.ceil(N_each*nproc/48), "ntasks": N_each*nproc,
                       "partition": 'skx-normal', "time": esti_time, "account": 'TG-EAR140030',
                      "mail-user": 'zhouton3@msu.edu', "mail-type": 'all'})
    s.run(thecommand)

In [3]:
# submit the job
import math
#evt_to_process=['200609101456A','200711262156A','200802211416A','200803151444A','200804180937A',
#               '200908031800A','201006150427A','201501282109A','201509130814A','201509241349A',
#               '201609031202A','201612280913A']
evt_to_process=open("evts.txt",'r').readlines()

# get dirs correct
nevt = len(evt_to_process)
for ievt in range(0,nevt):
    evt_to_process[ievt] = 'work/' + evt_to_process[ievt]
    
N_total = 30
N_each = 15
N_iter = 2
nproc = 256
comm = get_scripts(evt_to_process[0:30],N_each,N_iter,N_total,nproc)
print(comm)
esti_time = "1:40:00"
submit_job(comm,N_each,nproc,esti_time)

comm = get_scripts(evt_to_process[30:60],N_each,N_iter,N_total,nproc)
print(comm)
submit_job(comm,N_each,nproc,esti_time)

comm = get_scripts(evt_to_process[60:90],N_each,N_iter,N_total,nproc)
print(comm)
submit_job(comm,N_each,nproc,esti_time)

comm = get_scripts(evt_to_process[90:120],N_each,N_iter,N_total,nproc)
print(comm)
submit_job(comm,N_each,nproc,esti_time)

comm = get_scripts(evt_to_process[120:150],N_each,N_iter,N_total,nproc)
print(comm)
submit_job(comm,N_each,nproc,esti_time)

comm = get_scripts(evt_to_process[150:180],N_each,N_iter,N_total,nproc)
print(comm)
submit_job(comm,N_each,nproc,esti_time)

comm = get_scripts(evt_to_process[180:210],N_each,N_iter,N_total,nproc)
print(comm)
submit_job(comm,N_each,nproc,esti_time)

echo 'start xspecfem3D'; echo 'start iteration 0'; cd work/200505081707A
; ibrun -n 256 -o 0 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200506091400A
; ibrun -n 256 -o 256 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200506121541A
; ibrun -n 256 -o 512 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200506150251A
; ibrun -n 256 -o 768 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200506170621A
; ibrun -n 256 -o 1024 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200507260408A
; ibrun -n 256 -o 1280 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200509020127A
; ibrun -n 256 -o 1536 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200601040105A
; ibrun -n 256 -o 1792 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd 

submitted: b'-----------------------------------------------------------------\n          Welcome to the Stampede2 Supercomputer                 \n-----------------------------------------------------------------\n\nNo reservation for this job\n--> Verifying valid submit host (login4)...OK\n--> Verifying valid jobname...OK\n--> Enforcing max jobs per user...OK\n--> Verifying availability of your home dir (/home1/05880/tg851792)...OK\n--> Verifying availability of your work dir (/work/05880/tg851792/stampede2)...OK\n--> Verifying availability of your scratch dir (/scratch/05880/tg851792)...OK\n--> Verifying valid ssh keys...OK\n--> Verifying access to desired queue (skx-normal)...OK\n--> Verifying job request is within current queue limits...OK\n--> Checking available allocation (TG-EAR140030)...OK\nSubmitted batch job 3232145'


echo 'start xspecfem3D'; echo 'start iteration 0'; cd work/200709170016A
; ibrun -n 256 -o 0 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200709170804A
; ibrun -n 256 -o 256 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200710310304A
; ibrun -n 256 -o 512 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200712122340A
; ibrun -n 256 -o 768 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200801051101A
; ibrun -n 256 -o 1024 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200801051144A
; ibrun -n 256 -o 1280 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200801091440A
; ibrun -n 256 -o 1536 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200801100137A
; ibrun -n 256 -o 1792 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd 

submitted: b'-----------------------------------------------------------------\n          Welcome to the Stampede2 Supercomputer                 \n-----------------------------------------------------------------\n\nNo reservation for this job\n--> Verifying valid submit host (login4)...OK\n--> Verifying valid jobname...OK\n--> Enforcing max jobs per user...OK\n--> Verifying availability of your home dir (/home1/05880/tg851792)...OK\n--> Verifying availability of your work dir (/work/05880/tg851792/stampede2)...OK\n--> Verifying availability of your scratch dir (/scratch/05880/tg851792)...OK\n--> Verifying valid ssh keys...OK\n--> Verifying access to desired queue (skx-normal)...OK\n--> Verifying job request is within current queue limits...OK\n--> Checking available allocation (TG-EAR140030)...OK\nSubmitted batch job 3232146'


echo 'start xspecfem3D'; echo 'start iteration 0'; cd work/200901051059A
; ibrun -n 256 -o 0 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200905282132A
; ibrun -n 256 -o 256 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200905282257A
; ibrun -n 256 -o 512 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200905290104A
; ibrun -n 256 -o 768 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200906032137A
; ibrun -n 256 -o 1024 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200907030957A
; ibrun -n 256 -o 1280 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200907031100A
; ibrun -n 256 -o 1536 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/200907032030A
; ibrun -n 256 -o 1792 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd 

...

submitted: b'-----------------------------------------------------------------\n          Welcome to the Stampede2 Supercomputer                 \n-----------------------------------------------------------------\n\nNo reservation for this job\n--> Verifying valid submit host (login4)...OK\n--> Verifying valid jobname...OK\n--> Enforcing max jobs per user...OK\n--> Verifying availability of your home dir (/home1/05880/tg851792)...OK\n--> Verifying availability of your work dir (/work/05880/tg851792/stampede2)...OK\n--> Verifying availability of your scratch dir (/scratch/05880/tg851792)...OK\n--> Verifying valid ssh keys...OK\n--> Verifying access to desired queue (skx-normal)...OK\n--> Verifying job request is within current queue limits...OK\n--> Checking available allocation (TG-EAR140030)...OK\nSubmitted batch job 3232149'


echo 'start xspecfem3D'; echo 'start iteration 0'; cd work/201208262058A
; ibrun -n 256 -o 0 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201208272316A
; ibrun -n 256 -o 256 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201209252345A
; ibrun -n 256 -o 512 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201209281524A
; ibrun -n 256 -o 768 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201210052002A
; ibrun -n 256 -o 1024 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201210080626A
; ibrun -n 256 -o 1280 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201210210655A
; ibrun -n 256 -o 1536 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201210280304A
; ibrun -n 256 -o 1792 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd 

submitted: b'-----------------------------------------------------------------\n          Welcome to the Stampede2 Supercomputer                 \n-----------------------------------------------------------------\n\nNo reservation for this job\n--> Verifying valid submit host (login4)...OK\n--> Verifying valid jobname...OK\n--> Enforcing max jobs per user...OK\n--> Verifying availability of your home dir (/home1/05880/tg851792)...OK\n--> Verifying availability of your work dir (/work/05880/tg851792/stampede2)...OK\n--> Verifying availability of your scratch dir (/scratch/05880/tg851792)...OK\n--> Verifying valid ssh keys...OK\n--> Verifying access to desired queue (skx-normal)...OK\n--> Verifying job request is within current queue limits...OK\n--> Checking available allocation (TG-EAR140030)...OK\nSubmitted batch job 3232151'


echo 'start xspecfem3D'; echo 'start iteration 0'; cd work/201309150923A
; ibrun -n 256 -o 0 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201309211216A
; ibrun -n 256 -o 256 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201309211242A
; ibrun -n 256 -o 512 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201309211316A
; ibrun -n 256 -o 768 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201310191755A
; ibrun -n 256 -o 1024 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201312010319A
; ibrun -n 256 -o 1280 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201312200527A
; ibrun -n 256 -o 1536 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd work/201312231645A
; ibrun -n 256 -o 1792 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/initial_forward; cd 

submitted: b'-----------------------------------------------------------------\n          Welcome to the Stampede2 Supercomputer                 \n-----------------------------------------------------------------\n\nNo reservation for this job\n--> Verifying valid submit host (login4)...OK\n--> Verifying valid jobname...OK\n--> Enforcing max jobs per user...OK\n--> Verifying availability of your home dir (/home1/05880/tg851792)...OK\n--> Verifying availability of your work dir (/work/05880/tg851792/stampede2)...OK\n--> Verifying availability of your scratch dir (/scratch/05880/tg851792)...OK\n--> Verifying valid ssh keys...OK\n--> Verifying access to desired queue (skx-normal)...OK\n--> Verifying job request is within current queue limits...OK\n--> Checking available allocation (TG-EAR140030)...OK\nSubmitted batch job 3232153'
